In [ ]:
import pandas as pd

def _calculate_savings(time_to_complete, KS, KT, KH, r1, r2):
    AP = r1 * (KS / (KS + KT) + KT / (KS + KT))
    CP = 1 - r2 * ((KH - 1) / KH)
    savings_per_task_minutes = time_to_complete * AP * CP
    return savings_per_task_minutes

def return_on_investment(
    number_of_steps,
    number_of_tools,
    number_of_humans,
    frequency_per_month,
    time_to_complete_minutes
) -> dict[str:float]:

    savings_per_task_minutes = _calculate_savings(
        time_to_complete_minutes,
        number_of_steps,
        number_of_tools,
        number_of_humans,
        0.18,  # min_benchmarked_savings_rate
        0.38   # max_benchmarked_savings_rate
    )
    total_savings_per_month_minutes = savings_per_task_minutes * frequency_per_month
    percentage_decrease = (savings_per_task_minutes / time_to_complete_minutes) * 100

    return {
        "percentage_decrease": percentage_decrease,
        "hours_saved_per_month": total_savings_per_month_minutes / 60
    }

def calculate_savings_for_task_updated(row):
    results = return_on_investment(
        row["Steps"],  # number_of_steps
        row["Tools"],   # number_of_tools
        row["Humans"],  # number_of_humans
        row["Frequency"], # frequency_per_month
        row["Minutes"]  # time_to_complete_minutes
    )

    total_minutes_original = row["Time"]
    total_minutes_new = total_minutes_original - (results["hours_saved_per_month"] * 60)

    row["time saved by AI"] = results["hours_saved_per_month"] * 60
    row["time after AI"] = total_minutes_new

    return row

def analyze_tasks_from_csv_updated():
    # Ask for the filename
    filename = input("Please enter the filename of the CSV: ")

    # Load the CSV
    tasks_df = pd.read_csv(filename)

    # Process each task in the DataFrame
    tasks_df = tasks_df.apply(calculate_savings_for_task_updated, axis=1)

    # Sort tasks based on the time saved and take the top 3
    top_3_tasks = tasks_df.sort_values("time saved by AI", ascending=False).head(3)

    # Display the top 3 tasks
    print(top_3_tasks[["Task name", "Time", "time saved by AI", "time after AI"]])

    # Save the updated DataFrame with the new columns to a new CSV file
    output_filename = "tasks_with_savings.csv"
    tasks_df.to_csv(output_filename, index=False)
    print(f"\nThe updated data has been saved to: {output_filename}")

if __name__ == "__main__":
    analyze_tasks_from_csv_updated()
